In [1]:
import numpy as np
import transformers
transformers.logging.set_verbosity_error()
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict, Dataset, load_metric
from sklearn.metrics import confusion_matrix
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

In [2]:
from classes import RobertaForSequenceClassification2, StrIgnoreDevice, DataLoaderWithTaskname, MultitaskDataloader, MultitaskTrainer

In [3]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

def read_and_tokenize_data(train,test,evaluate):
    d = load_dataset('pandas', data_files={"train":train, "test":test,"evaluate":evaluate})
    for i in d:
        d[i]= d[i].remove_columns(column_names = ['__index_level_0__'])
    d = d.map(tokenize_function, batched=True)
    return d

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [4]:
dataset_dict = {
    "author":  read_and_tokenize_data("data/authors_train.pkl","data/authors_test.pkl","data/authors_validation.pkl"),
    "sentiment":  read_and_tokenize_data("data/imdb_train.pkl","data/imdb_test.pkl","data/imdb_validation.pkl")
}


multitask_model = RobertaForSequenceClassification2.from_pretrained(
        "roberta-base",
        task_labels_map={"author": 4, "sentiment": 2},
    )


for i in dataset_dict.values():
    for j in i.values():
        j.set_format(type="torch",columns=["input_ids", "attention_mask", "label"])
    
    
train_dataset = {
        task_name: dataset["train"] for task_name, dataset in dataset_dict.items()}

eval_dataset = {
        task_name: dataset["evaluate"] for task_name, dataset in dataset_dict.items()}

test_dataset = {
        task_name: dataset["test"] for task_name, dataset in dataset_dict.items()}

Using custom data configuration default-308df153f829b177
Reusing dataset pandas (/Users/mgaulia/.cache/huggingface/datasets/pandas/default-308df153f829b177/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /Users/mgaulia/.cache/huggingface/datasets/pandas/default-308df153f829b177/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade/cache-b02511b695f1c9e3.arrow
Loading cached processed dataset at /Users/mgaulia/.cache/huggingface/datasets/pandas/default-308df153f829b177/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade/cache-68a2a10ec55d40ad.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

Using custom data configuration default-0192cbfa45180d60
Reusing dataset pandas (/Users/mgaulia/.cache/huggingface/datasets/pandas/default-0192cbfa45180d60/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade)


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

#### Training the model

In [7]:
training_args = TrainingArguments(
    output_dir='results',
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=15,
    evaluation_strategy="steps",
    eval_steps = 500,
    save_steps = 500,
    load_best_model_at_end = True,
    per_device_eval_batch_size =  8
)


metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = MultitaskTrainer(
    model=multitask_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=training_args,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

In [7]:
trainer.train()

***** Running training *****
  Num examples = 5391
  Num Epochs = 15
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 10125
***** Running Evaluation *****
  Num examples = 300
  Batch size = 8


{'loss': 0.6787, 'learning_rate': 1.901234567901235e-05, 'epoch': 0.74}


Saving model checkpoint to results/checkpoint-500
Configuration saved in results/checkpoint-500/config.json


{'eval_loss': 0.5599084496498108, 'eval_accuracy': 0.8466666666666667, 'eval_runtime': 3.2378, 'eval_samples_per_second': 92.656, 'eval_steps_per_second': 11.736, 'epoch': 0.74}


Model weights saved in results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 300
  Batch size = 8


...

Saving model checkpoint to results/checkpoint-10000
Configuration saved in results/checkpoint-10000/config.json


{'eval_loss': 0.6036664843559265, 'eval_accuracy': 0.94, 'eval_runtime': 3.0673, 'eval_samples_per_second': 97.806, 'eval_steps_per_second': 12.389, 'epoch': 14.81}


Model weights saved in results/checkpoint-10000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from results/checkpoint-1500 (score: 0.37365713715553284).


{'train_runtime': 2821.0705, 'train_samples_per_second': 0.011, 'train_steps_per_second': 3.589, 'train_loss': 0.10714140081626397, 'epoch': 15.0}


TrainOutput(global_step=10125, training_loss=0.10714140081626397, metrics={'train_runtime': 2821.0705, 'train_samples_per_second': 0.011, 'train_steps_per_second': 3.589, 'train_loss': 0.10714140081626397, 'epoch': 15.0})

#### Saving the model

In [8]:
def save_model(trainer,tokenizer,trainer_path,tokenizer_path):
    trainer.save_model(trainer_path)
    tokenizer.save_pretrained(tokenizer_path)
    
save_model(trainer,tokenizer,"models/multihead_classification/model","models/multihead_classification/tokenizer")

Saving model checkpoint to models/multihead_classification/model
Configuration saved in models/multihead_classification/model/config.json
Model weights saved in models/multihead_classification/model/pytorch_model.bin
tokenizer config file saved in models/multihead_classification/tokenizer/tokenizer_config.json
Special tokens file saved in models/multihead_classification/tokenizer/special_tokens_map.json


#### Loading the model

In [8]:
def load_model(model_path,tokenizer_path):
    model = RobertaForSequenceClassification2.from_pretrained(model_path,task_labels_map={"author": 4, "sentiment": 2})
    tokenizer = RobertaTokenizer.from_pretrained(tokenizer_path)
    trainer = Trainer(model = model)
    return (model, trainer, tokenizer)
    

model, trainer, tokenizer = load_model("models/multihead_classification/model","models/multihead_classification/tokenizer")

loading configuration file models/multihead_classification/model\config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForSequenceClassification2"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.15.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file models/multihead_classification/model\pytorch_model.bin
All model checkpoint weights were used when initializing RobertaForSequenceClassification2.

All the weights of RobertaForSequenceC

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def make_predictions(test_data):
    metric = load_metric("accuracy")
    for task in ["author", "sentiment"]:
            y_true = []
            y_pred = []
            logits = model(input_ids = test_data[task]["input_ids"].to(device),\
                                     attention_mask = test_data[task]["attention_mask"].to(device),task_name=task)[0]
            y_true.append(i["label"])
            y_pred.append(np.argmax(logits.cpu().detach().numpy(), axis=1))
            print(confusion_matrix(y_true, y_pred))
            acc = metric.compute(predictions=y_pred, references=y_true)
            print(f"Task: {task}", f"Accuracy: {acc}")

In [45]:
def make_predictions(test_data):
    metric = load_metric("accuracy")
    for task in ["author", "sentiment"]:
            y_true = []
            y_pred = []
            for i in test_data[task]:
                logits = model(input_ids = torch.unsqueeze(i["input_ids"],0).to(device),\
                                         attention_mask = torch.unsqueeze(i["attention_mask"],0).to(device),task_name=task)[0]
                y_true.append(i["label"])
                y_pred.append(np.argmax(logits.cpu().detach().numpy(), axis=-1))
            print(confusion_matrix(y_true, y_pred))
            acc = metric.compute(predictions=y_pred, references=y_true)
            print(f"Task: {task}", f"Accuracy: {acc}")

In [48]:
make_predictions(test_dataset)

[[48  0  0  0]
 [ 1 20  0  0]
 [ 1  0  4  1]
 [ 2  8  0 64]]
Task: author Accuracy: {'accuracy': 0.912751677852349}
[[70  7]
 [ 4 69]]
Task: sentiment Accuracy: {'accuracy': 0.9266666666666666}


In [40]:
def pipeline(text, task):
    inputs = tokenizer(text,padding="longest",return_tensors="pt")
    inputs["input_ids"] =  inputs["input_ids"].to(device)
    inputs["attention_mask"] =  inputs["attention_mask"].to(device)
    logits = model(**inputs, task_name=task)["logits"]
    probabilities = torch.softmax(logits, dim=1).detach().cpu()
    result = torch.argmax(
        probabilities,axis=1
    )
    probability = torch.max(
        probabilities,axis=1
    )[0]
    print({"label":result, "score":probability})

In [42]:
pipeline("Total dissapointment. I do not understand why would anyone watch this","sentiment")

{'label': tensor([0]), 'score': tensor([0.9992])}


In [43]:
pipeline("An all time classic","sentiment")

{'label': tensor([1]), 'score': tensor([0.9981])}
